In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, median, when, lit, unix_timestamp, from_unixtime, min, max
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Initialize Spark Session
spark = SparkSession.builder.appName("Seismic_Events").getOrCreate()

# Load raw seismic data from Delta Table
usgs_df = spark.read.format("delta").table("tabular.dataexpert.usgs_seismic_events_silver")
iris_df = spark.read.format("delta").table("tabular.dataexpert.iris_seismic_events_silver")

#Display data before joining
#iris_df.show(5)
#usgs_df.show(5)
#usgs_df.count()




In [0]:
# Add a column to determine severity of earthquake based on the depth of the earthquake

usgs_df = usgs_df.withColumn(
    'severity',
    F.when(F.col('depth') < 70, 'High Severity')
     .when((F.col('depth') >= 70) & (F.col('depth') < 300), 'Moderate Severity')
     .otherwise('Low Severity')
)
#display(usgs_df)

In [0]:
# Iris dataset with distinct values of latitude, longitude and location
iris_filtered_df = iris_df.select(
    'latitude', 
    'longitude', 
    'event_location_name'
).distinct()



#  Join 'iris_df' and 'usgs_df' on latitude and longitude with precision up to 2 decimals
joined_df = usgs_df.alias('usgs') \
     .join(
        iris_filtered_df.alias('iris'),
        (F.col('iris.latitude') == F.col('usgs.latitude')) & 
        (F.col('iris.longitude') == F.col('usgs.longitude')),
        'left'
     ) \
    .select(
        'usgs.*',  # Select all columns from the usgs DataFrame
        'iris.event_location_name'
    )

final_df = joined_df.filter(F.col('event_location_name').isNotNull())
final_df.count()

In [0]:
#Rename event_location name column

final_df = final_df.withColumnRenamed("event_location_name", "location")
#display(final_df)

In [0]:
# find minimum and maximum magnitude for a region
final_df = final_df.withColumn(
    "min_region_magnitude", 
    F.min(F.col("magnitude")).over(Window.partitionBy("location")))

final_df = final_df.withColumn(
    "max_region_magnitude", 
    F.max(F.col("magnitude")).over(Window.partitionBy("location")))

final_df.show(2)

# final_df.select('magnitude', 'location', 'min_region_magnitude', 'max_region_magnitude').show(500)

In [0]:
#drop columns that are not being used

final_df = final_df.drop("detail", "timezone", "url", "ids", "types")


In [0]:
final_df.printSchema()

In [0]:
# Check if the table exists
if spark.catalog.tableExists("tabular.dataexpert.seismic_events_gold"):
    # Drop the table if it exists
    spark.sql("DROP TABLE tabular.dataexpert.seismic_events_gold")

# Write the new data to the Delta table
final_df.write.format("delta").mode("overwrite").partitionBy("year", "month").saveAsTable("tabular.dataexpert.seismic_events_gold")